
# Tutoriel Pyspark - Intro I
<p style = "font-size:100%"> Ce tutoriel d'introduction à Pyspark, introduit et présente les concepts suivants: spark context, spark sessions, spark catalog et global/local temporal view
</p>

### 1. Importation des librairies nécessaires

In [2]:
import os, time, psutil

In [3]:
# To run pypsark in notebooks: 
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

#### 2. Création de fonctions utiles

In [10]:
def get_system_info():
    # Get CPU information
    physical_cores = psutil.cpu_count(logical=False)
    total_cores = psutil.cpu_count(logical=True)
    
    # Get memory information
    memory = psutil.virtual_memory()
    total_memory = memory.total / (1024 ** 3)  # Convert to GB
    
    # Print information
    print(f"CPU Information:")
    print(f"  Physical cores: {physical_cores}")
    print(f"  Total cores (including logical): {total_cores}")
    print(f"\nMemory Information:")
    print(f"  Total memory: {total_memory:.2f} GB")
    
    # Get Spark-related environment variables
    spark_home = os.environ.get('SPARK_HOME', 'Not set')
    python_path = os.environ.get('PYTHONPATH', 'Not set')
    
    print(f"\nEnvironment Variables:")
    print(f"  SPARK_HOME: {spark_home}")
    print(f"  PYTHONPATH: {python_path}")

In [11]:
# We use SparkConf to set up the config options during the creation of the spark's session
def create_session_with_conf(app_name, master, configs=None):
    conf = SparkConf().setAppName(app_name).setMaster(master)
    
    if configs:
        for key, value in configs.items():
            conf.set(key, value)
    
    return SparkSession.builder \
        .config(conf=conf) \
        .enableHiveSupport() \
        .getOrCreate()


In [12]:
# We set directly the config options using .config() when creating the spark's session
def create_simple_session(app_name, master, cores, memory):
    return SparkSession.builder \
        .appName(app_name) \
        .master(master) \
        .config("spark.executor.cores", cores) \
        .config("spark.executor.memory", memory) \
        .enableHiveSupport() \
        .getOrCreate()

In [13]:
def display_session_info(session):
    print(f"Session ID: {session.sparkContext.applicationId}")
    print(f"Master: {session.sparkContext.master}")
    print(f"Executor Cores: {session.sparkContext.getConf().get('spark.executor.cores')}")
    print(f"Executor Memory: {session.sparkContext.getConf().get('spark.executor.memory')}")
    print(f"Shuffle Partitions: {session.sparkContext.getConf().get('spark.sql.shuffle.partitions')}")
    print(f"Default Parallelism: {session.sparkContext.getConf().get('spark.default.parallelism')}")
    print(f"Dynamic Allocation: {session.sparkContext.getConf().get('spark.dynamicAllocation.enabled')}")
    print("------------------")

### 3. Inspection des ressources locales disponibles

In [5]:
get_system_info()

CPU Information:
  Physical cores: 6
  Total cores (including logical): 12

Memory Information:
  Total memory: 15.81 GB

Environment Variables:
  SPARK_HOME: Not set
  PYTHONPATH: C:\Users\PMA831\Documents\jordan\FREELANCE\PernodRicard\PR-DEV\PR.Data.CleansingFactory.Transformer;C:\Users\PMA831\Documents\jordan\FREELANCE\PernodRicard\PR-DEV\PR.Data.CleansingFactory.Transformer


### 4. Création des sessions

Nb en 'local mode' il n'y a qu'un executor, puisque le master et le worker node sont la même machine, le driver et executor tourne sur la même machine 'local'.

In [14]:
session1 = create_simple_session("Session1", "local[2]", "2", "2g") # uses up to 2 (logical) cores, 2 core per executor, 2g memory per executor for this session
print("Session 1 Configuration:")
display_session_info(session1)

Session 1 Configuration:
Session ID: local-1728299401257
Master: local[2]
Executor Cores: 2
Executor Memory: 2g
Shuffle Partitions: None
Default Parallelism: None
Dynamic Allocation: None
------------------


In [15]:
# Create session2 using SparkConf
session2_configs = {
    "spark.executor.cores": "4",
    "spark.executor.memory": "4g",
    "spark.sql.shuffle.partitions": "8",
    "spark.default.parallelism": "8",
    "spark.dynamicAllocation.enabled": "false"
}
session2 = create_session_with_conf("Session2", "local[4]", session2_configs) 

print("\nSession 2 Configuration:")
display_session_info(session2)



Session 2 Configuration:
Session ID: local-1728299401257
Master: local[2]
Executor Cores: 2
Executor Memory: 2g
Shuffle Partitions: None
Default Parallelism: None
Dynamic Allocation: None
------------------


Pourquoi il n'a pas pris en compte ma configuration ? 
<p> >> Car les configurations liées aux ressources (comme le nombre de cœurs, la mémoire de l'exécuteur) sont définies au niveau du SparkContext! </p>

### 5. Opérations

Le but est de voir que chaque session spark effectue les calculs de manière isolé de l'autre, et qu'en fonction des ressources mises à disposition, elles n'auront pas les mêmes durées d'exécution.

In [8]:
def perform_computation(session, num_partitions):
    """Prend une session en entrée, s'appuie sur le sparkContext (qui est unique pour l'ensemble des sessions) pour créer un RDD et effectue des opérations dessus"""
    large_range = session.sparkContext.parallelize(range(10000000), num_partitions)
    start_time = time.time()
    result = large_range.map(lambda x: x * x).sum()
    end_time = time.time()
    print(f"Sum of squares: {result}")
    print(f"Computation time: {end_time - start_time:.2f} seconds")


In [16]:
print("\nPerforming computation in Session 1:")
perform_computation(session1, 2)

print("\nPerforming computation in Session 2:")
perform_computation(session2, 4)


Performing computation in Session 1:


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 0.0 failed 1 times, most recent failure: Lost task 1.0 in stage 0.0 (TID 1) (W0010SL00066699.green.local executor driver): java.io.IOException: Cannot run program "C:\Users\PMA831\AppData\Local\Programs\Python\Python38": CreateProcess error=5, Accès refusé
	at java.base/java.lang.ProcessBuilder.start(ProcessBuilder.java:1128)
	at java.base/java.lang.ProcessBuilder.start(ProcessBuilder.java:1071)
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:167)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:164)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.sca

### 6. Views and catalog

In [ ]:
# Create DataFrames in each session
df1 = session1.createDataFrame([(1, "Alice"), (2, "Bob"), (3, "Charlie")], ["id", "name"])
df2 = session2.createDataFrame([(1, "David"), (2, "Eve"), (3, "Frank")], ["id", "name"])

##### 6.1) Local Temp View

In [ ]:
# Register temporary views
df1.createOrReplaceTempView("people_session1")
df2.createOrReplaceTempView("people_session2")

Par défaut, une vue temporaire n'existe que dans la session dans laquelle elle a été crée

In [ ]:
print("\nData in Session 1:")
session1.sql("SELECT * FROM people_session1").show()

In [ ]:
print("\nData in Session 2:")
session2.sql("SELECT * FROM people_session2").show()

Impossible donc d'accéder à une vue d'une autre session

In [ ]:
# De la session 1 on essaye d'accéder à la vue (crée dans la session 2)
session1.sql("SELECT * FROM people_session2").show()

In [ ]:
# De la session 2 on essaye d'accéder à la vue (crée dans la session 1)
session2.sql("SELECT * FROM people_session1").show()

##### 6.2) Global Temp View & catalog

Pour rendre une vue accessible à l'ensemble des sessions, on peut utiliser le Global Catalog

In [ ]:
# Global Catalog Usage

# 1. Global Temporary View
print("\n--- Global Temporary View ---")
df1.createGlobalTempView("global_people") # We register df1 as a global temp view in the global catalog

Les 2 sessions peuvent accéder à global_people

In [ ]:
print("Accessing global temporary view from Session 1:")
session1.sql("SELECT * FROM global_temp.global_people").show()

print("Accessing global temporary view from Session 2:")
session2.sql("SELECT * FROM global_temp.global_people").show()

##### 6.3) Spark SQL Warehouse directory

On peut aussi inscrire la donnée en "base" pour la partager entre plusieurs spark sessions

In [ ]:
# 2. Permanent Table in a Database
print("\n--- Permanent Table in Database ---")
session1.sql("CREATE DATABASE IF NOT EXISTS shared_db")
df1.write.mode("overwrite").saveAsTable("shared_db.permanent_people")

In [ ]:
print("Accessing permanent table from Session 1:")
session1.sql("SELECT * FROM shared_db.permanent_people").show()

In [ ]:
print("Accessing permanent table from Session 2:")
session2.sql("SELECT * FROM shared_db.permanent_people").show()

### 7. Opérations disponibles sur le catalog

In [ ]:
# Catalog Operations
print("\n--- Catalog Operations ---")

print("Databases in Session 1:")
session1.catalog.listDatabases()

print("\nTables in 'shared_db' from Session 1:")
session1.catalog.listTables("shared_db")

In [ ]:

print("\nDatabases in Session 2:")
session2.catalog.listDatabases()

print("\nTables in 'shared_db' from Session 2:")
session2.catalog.listTables("shared_db")

# Clean up
session1.catalog.dropGlobalTempView("global_people")
session1.sql("DROP TABLE IF EXISTS shared_db.permanent_people")
session1.sql("DROP DATABASE IF EXISTS shared_db")

### 8. Fin de session

In [ ]:
session1.stop()

In [ ]:
session2.stop()